# Amostragem

**Amostragem é uma contagem ou medição de parte da população e é usada em estudos estatísticos**

Observação: Para coletar dados não tendenciosos, você
deve ter certeza de que a amostra represente a população

## Carregamento da base de dados

In [1]:
import pandas as pd
import random
import numpy as np

In [2]:
dataset = pd.read_csv('credit_data.csv')

In [3]:
dataset.shape

(2000, 5)

In [4]:
dataset.head()

,i#clientid,income,age,loan,c#default
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0
3,4,42709.534201,45.751972,6103.642260,0
4,5,66952.688845,18.584336,8770.099235,1


In [5]:
dataset.tail()

,i#clientid,income,age,loan,c#default
1995,1996,59221.044874,48.518179,1926.729397,0
1996,1997,69516.127573,23.162104,3503.176156,0
1997,1998,44311.449262,28.017167,5522.786693,1
1998,1999,43756.056605,63.971796,1622.722598,0
1999,2000,69436.579552,56.152617,7378.833599,0


## Amostragem aleatória simples

In [6]:
def amostragem_aleatoria_simples(dataset, amostras):
  return dataset.sample(n = amostras, random_state=1)

In [7]:
df_amostra_aleatoria_simples = amostragem_aleatoria_simples(dataset, 1000)
df_amostra_aleatoria_simples.shape

(1000, 5)

## Amostragem sistemática

In [8]:
def amostragem_sistematica(dataset, amostras):
  intervalo = len(dataset) // amostras
  random.seed(1)
  inicio = random.randint(0, intervalo)
  indices = np.arange(inicio, len(dataset), step = intervalo)
  amostra_sistematica = dataset.iloc[indices]
  return amostra_sistematica

In [9]:
df_amostra_sistematica = amostragem_sistematica(dataset, 1000)
df_amostra_sistematica.shape

(1000, 5)

## Amostragem por grupos

In [10]:
def amostragem_agrupamento(dataset, numero_grupos):
  intervalo = len(dataset) / numero_grupos

  grupos = []
  id_grupo = 0
  contagem = 0
  for _ in dataset.iterrows():
    grupos.append(id_grupo)
    contagem += 1
    if contagem > intervalo:
      contagem = 0
      id_grupo += 1

  dataset['grupo'] = grupos
  random.seed(1)
  grupo_selecionado = random.randint(0, numero_grupos)
  return dataset[dataset['grupo'] == grupo_selecionado]

In [11]:
len(dataset) / 2

1000.0

In [12]:
df_amostra_agrupamento = amostragem_agrupamento(dataset, 2)
df_amostra_agrupamento.shape, df_amostra_agrupamento['grupo'].value_counts()

((1001, 6),
 grupo
 0    1001
 Name: count, dtype: int64)

## Amostra estratificada

In [13]:
from sklearn.model_selection import StratifiedShuffleSplit

In [14]:
dataset['c#default'].value_counts()

c#default
0    1717
1     283
Name: count, dtype: int64

In [15]:
def amostragem_estratificada(dataset, percentual, campo):
  split = StratifiedShuffleSplit(test_size=percentual, random_state=1)
  for _, y in split.split(dataset, dataset[campo]):
    df_y = dataset.iloc[y]
  return df_y

In [16]:
df_amostra_estratificada = amostragem_estratificada(dataset, 0.5, 'c#default')
df_amostra_estratificada.shape

(1000, 6)

## Comparativo dos resultados

In [17]:
media_idade = dataset['age'].mean(); media_renda = dataset['income'].mean(); media_emprestimo = dataset['loan'].mean()

In [18]:
media_idade_aleatoria = df_amostra_aleatoria_simples['age'].mean(); media_renda_aleatoria = df_amostra_aleatoria_simples['income'].mean();  media_emprestimo_aleatoria = df_amostra_aleatoria_simples['loan'].mean()

In [19]:
media_idade_sistemica = df_amostra_sistematica['age'].mean(); media_renda_sistemica = df_amostra_sistematica['income'].mean(); media_emprestimo_sistemica = df_amostra_sistematica['loan'].mean()

In [20]:
media_idade_agrupamento = df_amostra_agrupamento['age'].mean(); media_renda_agrupamento = df_amostra_agrupamento['income'].mean(); media_emprestimo_agrupamento = df_amostra_agrupamento['loan'].mean()

In [21]:
media_idade_estratificada = df_amostra_estratificada['age'].mean(); media_renda_estratificada = df_amostra_estratificada['income'].mean(); media_emprestimo_estratificada = df_amostra_estratificada['loan'].mean()

**Dica: faça a subtração dos valores dos atributos ("age", "income", "loan") do dataset original com os dos atributos de cada tipo de amostragem. A técnica a ser utilizada por você é aquela que tiver as menores diferenças.**

In [22]:
# usando amostragem aleatória simples

abs(media_idade - media_idade_aleatoria), abs(media_renda - media_renda_aleatoria), abs(media_emprestimo - media_emprestimo_aleatoria)

(0.31203376716028686, 231.66863597580232, 5.077205754107126)

In [23]:
# usando amostragem sistemica

abs(media_idade - media_idade_sistemica), abs(media_renda - media_renda_sistemica), abs(media_emprestimo - media_emprestimo_sistemica)

(0.10361443301296447, 359.89873287617957, 62.41828173807153)

In [24]:
# usando amostragem agrupamento

abs(media_idade - media_idade_agrupamento), abs(media_renda - media_renda_agrupamento), abs(media_emprestimo - media_emprestimo_agrupamento)

(0.23566373364572257, 484.85075793183205, 54.208200944053715)

In [25]:
# usando amostragem estratificada

abs(media_idade - media_idade_estratificada), abs(media_renda - media_renda_estratificada), abs(media_emprestimo - media_emprestimo_estratificada)

(0.27392230666538353, 230.23326752590947, 20.625043594063754)

In [26]:
data = [[media_idade, media_idade_aleatoria, media_idade_sistemica, media_idade_agrupamento, media_idade_estratificada],
        [media_renda, media_renda_aleatoria, media_renda_sistemica, media_renda_agrupamento, media_renda_estratificada],
        [media_emprestimo, media_emprestimo_aleatoria, media_emprestimo_sistemica, media_emprestimo_agrupamento, media_emprestimo_estratificada]
        ]

# Médias do dataset original e dos diferentes tipos de amostragens
columns = ["Original", "Aleatória", "Sistêmica", "Agrupamento", "Estratificada"]
indexes = ["idade", "renda", "emprestimo"]

df_medias = pd.DataFrame(data, columns=columns, index=indexes, dtype=float)

In [27]:
df_medias

,Original,Aleatória,Sistêmica,Agrupamento,Estratificada
idade,40.807559,40.495526,40.911174,41.043223,40.533637
renda,45331.600018,45563.268654,45691.498751,44846.749260,45101.366750
emprestimo,4444.369695,4449.446900,4506.787976,4390.161494,4423.744651


Melhor tipo de amostragem:

Age (Idade): Amostra sistêmica
Income (Renda): Amostragem estratificada
Loan (Empréstimo): Amostragem aleatória simples